In [39]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from ScraperFC import ClubElo

# Load data and put into a Pandas dataframe

The JSON files used in this notebook are available on [Google Drive](https://drive.google.com/drive/folders/1LhW3wcG5uoAAHcgPHRcJYmIQaMY9GlRI). They have been pre-scraped using the [WhoScored](https://www.whoscored.com/) module from [ScraperFC](https://github.com/oseymour/ScraperFC).

In [ ]:
datafiles = [
    #'Argentina_Liga_Profesional_2021_match_data.json', # not on clubelo
    #'Brasileirao_2021_match_data.json', # not on clubelo
#     'Bundesliga_2021_match_data.json',
    'Bundesliga_II_2021_match_data.json',
    'EFL_Championship_2021_match_data.json',
    #'EFL1_2021_match_data.json', # not on clubelo
    #'EFL2_2021_match_data.json', # not on clubelo
    'EPL_2021_match_data.json',
    'Eredivisie_2021_match_data.json',
    'Jupiler_Pro_League_2021_match_data.json',
    'La_Liga_2021_match_data.json',
    'Liga_Nos_2021_match_data.json',
    'Ligue_1_2021_match_data.json',
    #'MLS_2021_match_data.json', # not on clubelo
    'Russian_Premier_League_2021_match_data.json',
    'Serie_A_2021_match_data.json',
    'Super_Lig_2021_match_data.json',
]
df = pd.DataFrame()

for datafile in datafiles:
    
    league_name = datafile.split('_2021')[0].replace('_',' ')
    
    print('Parsing data for {}'.format(league_name))
    with open(datafile,'r') as f:
        match_data = json.loads(f.read())
        
    break
#     del match_data

Parsing data for Bundesliga II


In [58]:
team_names = {
    'RBL': 'RBLeipzig',
    'Werder Bremen': 'Werder',
    'FC Koln': 'Koeln',
    'Borussia M.Gladbach': 'Gladbach',
    'Eintracht Frankfurt': 'Frankfurt',
    'Borussia Dortmund': 'Dortmund',
    'Union Berlin': 'UnionBerlin',
    'Arminia Bielefeld': 'Bielefeld',
    'Hertha Berlin': 'Hertha'
}

df = pd.DataFrame()
for i, match in enumerate(match_data.values()):
    
    print('{}/{}'.format(i+1, len(match_data.values())), end='\r')
    
    new_row = pd.Series(dtype=object)
    new_row['League'] = league_name
    # team names
    home_team = match['matchCentreData']['home']['name']
    away_team = match['matchCentreData']['away']['name']
    new_row['Home team'] = home_team
    new_row['Away team'] = away_team
    # ELO scores
    match_date = match['matchCentreData']['startDate'].split('T')[0]
    home_elo = ClubElo().scrape_team_on_date(home_team, match_date)
    away_elo = ClubElo().scrape_team_on_date(away_team, match_date)
    if home_elo == -1:
        home_elo = ClubElo().scrape_team_on_date(team_names[home_team], match_date)
    if away_elo == -1:
        away_elo = ClubElo().scrape_team_on_date(team_names[away_team], match_date)
    assert home_elo!=-1 and away_elo!=-1
    new_row['Home ELO'] =  home_elo
    new_row['Away ELO'] = away_elo
    # formations
    new_row['Home formation'] = match['matchCentreData']['home']['formations'][0]['formationName']
    new_row['Away formation'] = match['matchCentreData']['away']['formations'][0]['formationName']
    # score and result
    score = match['matchCentreData']['score']
    home_goals = int(score.split(' : ')[0])
    away_goals = int(score.split(' : ')[1])
    new_row['Home goals'] = home_goals
    new_row['Away goals'] = away_goals
    if home_goals > away_goals: # options are HW (home win), AW (away win), D (draw)
        new_row['Result'] = 'hw'
    elif home_goals < away_goals:
        new_row['Result'] = 'aw'
    else:
        new_row['Result'] = 'd'
        
    df = df.append(new_row, ignore_index=True)

<ipython-input-58-86145cc99dea>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_row = pd.Series()


KeyboardInterrupt: 

In [ ]:
df

# Win rate vs. other formations

# Goals scored against other formations

# Goals conceded against other formations